In [8]:
%pylab
from star_cluster import StarCluster
from filterlists import * 
#from slugpy.cluster_slug import cluster_slug

#cs = cluster_slug(use_nebular=False,photsystem="Vega",libname="CLUSTERSLUG_MW")
#cs.add_filters(messa_m51_filters)

# for i in range(10):
#     sc = StarCluster(10**4,model="EFF",background=1e-2,rmax=30,scale_radius=1.0,shape=2.5)
#     plt.plot(np.linspace(6,9,101),[sc.measure_r50(count_photons=True,age=a,method='poisson') for a in np.logspace(6,9,101)]); plt.yscale('log')
#phot = sc.get_photometry(3e8,["ACS_F555W"])
#plt.scatter(np.log10(sc.masses),phot[:,0])
#plt.scatter(np.log10(sc.masses),phot[:,1])
#phot = sc.get_photometry(1e7)
#plt.plot(np.log10(np.sort(sc.masses)), phot[sc.masses.argsort()])
#sc.measure_bayesian(1e6,"Mass",cs=cs)

#rc, rb = sc.get_all_radii()
#sc.get_background_radii(),sc.get_cluster_radii()
#%timeit sc.fit_density_profile() #full_output=True)



Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/mgrudic/.local/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['vectorize', 'gamma']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [9]:
from slugpy.cluster_slug import cluster_slug
import numpy as np
cs = cluster_slug(use_nebular=False,photsystem="Vega")
cs.add_filters(messa_m51_filters)

In [18]:
#from random_cluster_population import *
from density_models import *
import numpy as np
from star_cluster import StarCluster
from filterlists import * 

#ML_age, ML = np.loadtxt("Bruzual_Charlot_2003_ML_vs_Age.csv").T

data = []
#data = list(np.load("mass_radius_measurements.npy"))
slopes = []
#for i in range(1):
while True:
    #masses, ages, gammas = random_cluster_population(100,Mmin=5e3)
    Mmin, Mmax = 2e3, 1e6
    tmin, tmax = 3e6,1e9
    nclusters = 100
    gammas = np.random.uniform(2.2,3.2,nclusters)
    x = np.random.rand(nclusters)
    true_masses = Mmax * Mmin / (Mmax * (1 - x) + Mmin * x)
    ages = 10**(np.log10(tmin)+np.log10(tmax/tmin)*np.random.rand(nclusters))
    r50s = []
    masses = []
    true_radii = []
    tm = []
    ta = []
    measured_ages = []
    #true_masses =[]
    
    for mass, age, gamma in zip(true_masses,ages,gammas):
        #print(mass)
        numstars = int(mass/0.5+0.5)
        sc = StarCluster(num_stars=numstars, scale_radius=3*(mass/1e4)**(1./3)/model_r50(gamma,1.,'EFF'), shape=gamma, background=1e-1,model='EFF')
        measured_mass = 10**sc.measure_bayesian(age,"Mass",cs=cs)[1]
        #if measured_mass < 5e3: continue
        measured_age = 10**sc.measure_bayesian(age,"Age",cs=cs)[1]
        #if measured_age > 3e8 or measured_age < 1e7: continue
        masses.append(measured_mass)
        r50s.append(sc.measure_r50(count_photons=True,age=age))
        tm.append(mass)
        ta.append(age)
        true_radii.append(3*(mass/1e4)**(1./3))
        measured_ages.append(measured_age)
    cut = np.isfinite(r50s) * (np.array(r50s)<100) * (np.array(r50s) > 0.1)
    slopes.append(np.polyfit(np.log10(masses)[cut],np.log10(r50s)[cut],1)[0])
    data.append([true_masses,true_radii,ages,gammas,masses,r50s,measured_ages])
    np.save("mass_radius_measurements.npy",data)
    print(slopes[-1])
print(np.median(slopes),np.std(slopes))

/home/mgrudic/code/cluster_profile_sim/lossfuncs.py:45: RuntimeWarning: invalid value encountered in divide
  mu
/home/mgrudic/code/cluster_profile_sim/density_models.py:58: RuntimeWarning: overflow encountered in scalar power
  return np.sqrt(2 ** (2 / (gamma - 2)) - 1) * scale_radius
/home/mgrudic/code/cluster_profile_sim/star_cluster.py:300: OptimizeWarning: Initial guess is not within the specified bounds
  sol = minimize(


0.19430633293928024
0.28472135893499506
0.2663622418974146
0.2112036473194238
0.26064739163372624
0.268063848179964
0.3143661616913643
0.3471029628502346
0.2253989745833939
0.17136851398398376
0.17592202412317867
0.2064289898944529
0.21433331444633932
0.2749334249145903
-0.027376185100030946
0.1950110112199277
0.19725329979820647
0.3441157002049727
0.27723578580952807
0.2515028919552688
0.2960090887634017
0.22856847099830158
0.1935546488372776
0.4290398486743714
0.2577821926807954
0.04184253331995193
0.2976191730882844
0.1967641316406711
0.3386121475828918
0.12823728304068555
0.28276892470493503
0.23176660025415105
0.24537266452560613
0.20120517113988323
0.28777179564671757
0.13704294145119197
0.19699594921584435
0.2185434540099219
0.1354424931507026
0.2724869562255779
0.15200478469604597
0.1853029870450675
0.12030074652583704
0.265823031145038
0.2014327128288798
0.16902647030161316
0.06235015896582666
0.2674111328243077
0.1851077465836338
0.4037857320495876
0.2583954651908075
0.250224

KeyboardInterrupt: 

In [21]:
len(data[0])

7

In [22]:
np.load("mass_radius_measurements.npy").shape
#data.shape

(1028, 7, 100)

In [36]:
#sc.measure_bayesian(age,"Mass",cs=cs)
plt.figure(figsize=(4,4))
plt.loglog(masses,r50s,'.',label="Measured")
plt.loglog(tm,3*(np.array(tm)/1e4)**(1./3),'.',label="True")
for i in range(len(tm)):
    plt.loglog([tm[i],masses[i]],[3*(tm[i]/1e4)**(1./3),r50s[i]],color='black',alpha=0.3)
plt.xlabel(r"$M\,\left(M_\odot\right)$")
plt.ylabel(r"$R_{\rm eff}\,\left(\rm pc\right)$")
plt.legend()
cut = np.isfinite(r50s) * (np.array(r50s)<100) * (np.array(r50s) > 0.1)
#print(cut.sum())
fit = np.polyfit(np.log10(masses)[cut],np.log10(r50s)[cut],1)
logr_model = np.polyval(fit, np.log10(masses)[cut])
np.std(logr_model - np.log10(r50s)[cut]), fit[0]


(0.26098949319320214, 0.4301914103045154)

In [23]:
np.polyval(fit, np.log10(masses)[cut])

TypeError: iteration over a 0-d array

In [4]:
import pickle
f
filt, track, grids = pickle.load(
    open(f"lumgrids/mass_lum_grid_{filt}_{track}.dump", "rb")
)
mgrid, Lgrid = grids[age_yr]
logmgrid = np.log10(mgrid)
logLgrid = np.log10(Lgrid)
mmax = mgrid[np.isfinite(logLgrid)].max()

NameError: name 'filt' is not defined

In [5]:
sc.num_stars

100000